# Medianas - como NO debería ser
Implementación de la generación del compuesto temporal de medianas, sobre el cubo V2, utilizando las consultas de bajo nivel provistas por el cubo (no paraleliza, ni utliza las funciones definidas en los componentes GridWorkflow, Analytics ni Execution)
## Parámetros

In [ ]:
longs=(-75, -74)
lats=(5, 6)
normalized=True
time_range=("2000-01-01","2000-12-31")
bands=["blue","green","red","nir", "swir1","swir2"]
#bands=["blue"]
minValid=1;

## Consulta

In [ ]:
%matplotlib inline
import datacube
import numpy as np
dc = datacube.Datacube(app='dc-prueba1')
nbar = dc.load(product='ls7_ledaps_utm', longitude=longs, latitude=lats, time=time_range)

In [ ]:
## Opcional (para ver los resultados en el cuaderno de python)
nbar

## Medianas:

In [ ]:
nodata=-9999
medians={}
for band in bands:
    
    datos=np.where(nbar.data_vars[band]!=nodata,nbar.data_vars[band], np.nan)
    allNan=~np.isnan(datos)
    if normalized:
        m=np.nanmean(datos.reshape((datos.shape[0],-1)), axis=1)
        st=np.nanstd(datos.reshape((datos.shape[0],-1)), axis=1)
        datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)
    medians[band]=np.nanmedian(datos,0)
    medians[band][np.sum(allNan,0)<minValid]=np.nan
del datos    

In [ ]:
# Opcional (para ver los resultados en el cuaderno de python)
medians

## Guardar los resultados a un archivo NetCDF

In [ ]:
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
nco=netcdf_writer.create_netcdf("salida2000.nc")
cords=('latitude', 'longitude','time')
for x in cords:
    if(x!="time"):
        netcdf_writer.create_coordinate(nco, x, nbar.coords[x].values, nbar.coords[x].units)
    else:
        netcdf_writer.create_coordinate(nco, x, np.array(["t1-t2"]), "period")
netcdf_writer.create_grid_mapping_variable(nco, nbar.crs)
for band in bands:
    medians[band][np.isnan(medians[band])]=nodata
    var= netcdf_writer.create_variable(nco, band, Variable(np.dtype(np.int32), None, ('time','latitude', 'longitude'), None) ,set_crs=True)
    var[:] = netcdf_writer.netcdfy_data(medians[band])
nco.close()